# Vessel Counts
You should look at the `voyages_routes` table when you want information about where a vessel was _and_ when it was there. This table is useful when you want to find how many vessels were in a specific region of the South China Sea on January 1, 2020.  To do this, you use a simple `count` aggregation after you filter your data:

In [1]:
from descarteslabs.vektorius import vector
from shapely.geometry import shape
import warnings
warnings.filterwarnings('ignore')

aoi = shape({
    "type": "Polygon",
    "coordinates": [
        [
            [104.9853515625, 11.049038346537106],
            [124.1455078125, 11.049038346537106],
            [124.1455078125, 24.766784522874453],
            [104.9853515625, 24.766784522874453],
            [104.9853515625,11.049038346537106]
        ]
    ]
})
route = vector.table("voyages_routes")

`voyages_routes` is a pretty large table, with over 150 million rows, which means interacting with  it can be cumbersom if you're not careful about how you filter your data.  When working with this table it's especially important to use temporal `Filters` whenever possible, because the `timestamp` column is Clustered, and temporal filters can dramatically reduce the amount of data that needs to be scanned.

In [2]:
# find routes that intersect with the AOI at the given point in time
filter_ = (route.timestamp.date() == "2020-01-01") & route.position.intersects(aoi)
filtered_route = route.filter(filter_)

# only grab distinct mmsi -representing the unique vessel identifier
filtered_route = filtered_route[filtered_route.mmsi].distinct()

# use the `count` aggregation to get the final count
# Note that the result is a simple scalar int
# because there's only 1 column and 1 row
vessel_count = filtered_route.count().execute()
vessel_count

67

You can make this aggregation a little more interesting by adding in some vessel information to the counts. You first Join vessel information from vessels to the filtered route information you grabbed before.

Joins work by specifying a left and right table, a Join type (`inner_join`, `left_join`, `cross_join`, etc), and a Join condition.

In [3]:
vessel = vector.table("vessels")

# `filtered_route` is the left table, that we want to inner_join
# to `vessel` where the `mmsi` column of both tables match
# only rows that appear in both tables will be returned
join_condition = vessel.mmsi == filtered_route.mmsi
joined = filtered_route.inner_join(vessel, join_condition)

# once you make a join, you need to select which columns you want
# returned from each table.  This needs to happen immediately after
# the join
data = joined[filtered_route.mmsi, vessel.ship_type]

Now that you've performed the Join you can construct your aggregation, which uses a more complex form of aggregation that specifies which columns you want to group by, and what aggregation algorithm you want to use. In this case, you want to see how many voyages were made by each `ship_type`, so you need to group by `ship_type` and get a count of each voyage.

In [4]:
# group by class, and specify the aggretation algorithm for the group
data = data.group_by(data.ship_type).aggregate(data.mmsi.count())

# this time, vessel_count is a `DataFrame` not a `GeoDataFrame`
# we have multiple columns and rows, but none of the columns are
# a Geospatial data type, so a `GeoDataFrame` isn't necessary
vessel_count = data.execute()
vessel_count

,ship_type,count
0,Cargo,65
1,Wing In Ground,1
2,Other,1
